In [38]:
import numpy as np
import pandas as pd
import re

## Import Files

In [50]:
nb = pd.read_csv("laptops.csv", encoding = 'unicode_escape')

In [51]:
import os
benchmark_files = os.path.join(os.getcwd(), "benchmarks")
def get_benchmark_df(file_name): return  pd.read_csv(os.path.join(benchmark_files, file_name), encoding = 'unicode_escape')
cpu = get_benchmark_df("CPU_UserBenchmarks.csv")
gpu = get_benchmark_df("GPU_UserBenchmarks.csv")

## Identify Performance

In [80]:
nb.head()

,Unnamed: 0,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price_euros
0,1,Apple,MacBook Pro,Ultrabook,13.3,IPS Panel Retina Display 2560x1600,Core i5,8GB,128GB SSD,Intel Iris Plus Graphics 640,macOS,1.37kg,1339.69
1,2,Apple,Macbook Air,Ultrabook,13.3,1440x900,Core i5,8GB,128GB Flash Storage,Intel HD Graphics 6000,macOS,1.34kg,898.94
2,3,HP,250 G6,Notebook,15.6,Full HD 1920x1080,Core i5 7200U,8GB,256GB SSD,Intel HD Graphics 620,No OS,1.86kg,575.00
3,4,Apple,MacBook Pro,Ultrabook,15.4,IPS Panel Retina Display 2880x1800,Core i7,16GB,512GB SSD,AMD Radeon Pro 455,macOS,1.83kg,2537.45
4,5,Apple,MacBook Pro,Ultrabook,13.3,IPS Panel Retina Display 2560x1600,Core i5,8GB,256GB SSD,Intel Iris Plus Graphics 650,macOS,1.37kg,1803.60


### CPU Benchmarks

In [53]:
cpu.head()

,Type,Part Number,Brand,Model,Rank,Benchmark,Samples,URL
0,CPU,BX8071512900KS,Intel,Core i9-12900KS,1,120.0,9,https://cpu.userbenchmark.com/SpeedTest/182151...
1,CPU,BX8071512900K,Intel,Core i9-12900K,2,114.0,19382,https://cpu.userbenchmark.com/Intel-Core-i9-12...
2,CPU,BX8071512900KF,Intel,Core i9-12900KF,3,113.0,3963,https://cpu.userbenchmark.com/SpeedTest/168558...
3,CPU,BX8071512700K,Intel,Core i7-12700K,4,112.0,22631,https://cpu.userbenchmark.com/Intel-Core-i7-12...
4,CPU,BX8071512900F,Intel,Core i9-12900F,5,112.0,82,https://cpu.userbenchmark.com/SpeedTest/175783...


In [54]:
nb[nb['Cpu'].str.contains('Ryzen')].head()

,Unnamed: 0,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price_euros
47,48,Asus,Rog Strix,Gaming,17.3,Full HD 1920x1080,AMD Ryzen 1700 3GHz,8GB,256GB SSD + 1TB HDD,AMD Radeon RX 580,Windows 10,3.2kg,1299.0
271,276,Asus,Rog Strix,Gaming,17.3,Full HD 1920x1080,AMD Ryzen 1700 3GHz,16GB,256GB SSD + 1TB HDD,AMD Radeon RX 580,Windows 10,3.2kg,1549.0
372,378,Asus,Rog Strix,Gaming,17.3,IPS Panel Full HD 1920x1080,AMD Ryzen 1700 3GHz,16GB,256GB SSD + 1TB HDD,AMD Radeon RX 580,Windows 10,3.25kg,2199.0
435,442,Asus,Rog Strix,Gaming,17.3,Full HD 1920x1080,AMD Ryzen 1600 3.2GHz,8GB,256GB SSD + 1TB HDD,AMD Radeon RX 580,Windows 10,3.2kg,1695.0


In [55]:
# Use aggregate scores for all of a given type if the actual model is not available

In [56]:
cpu[cpu['Model'].str.contains('7200')].head()


,Type,Part Number,Brand,Model,Rank,Benchmark,Samples,URL
589,CPU,NaN,Intel,Core i5-7200U,590,57.6,212369,https://cpu.userbenchmark.com/SpeedTest/153577...
1028,CPU,NaN,AMD,A8-7200P APU (2014 M.Ka),1029,39.4,130,https://cpu.userbenchmark.com/SpeedTest/64755/...
1032,CPU,NaN,Intel,Core2 Duo E7200,1033,39.2,5843,https://cpu.userbenchmark.com/SpeedTest/1876/I...
1181,CPU,NaN,Intel,Core2 T7200,1182,33.6,2298,https://cpu.userbenchmark.com/SpeedTest/248/In...


In [67]:
# Get CPU columns into the same format 
nb['Cpu'] = nb['Cpu'].astype('str')
cpu["Model"] = cpu["Model"].astype('str')
nb["Cpu"] = nb["Cpu"].apply(lambda x: re.sub("AMD|Intel|([\d\.]+GHz)|(Dual Core)|(Quad Core)|", "", x.strip()))
cpu["Model"] = cpu["Model"].apply(lambda x: re.sub("-", " ", x.strip()))

In [68]:
cpu_benchmarks = cpu[["Model","Benchmark"]]

In [78]:
nb_merge = nb.merge(cpu_benchmarks, left_on="Cpu",right_on="Model")#, how='left')
# Ignore all items which don't have clearly specified CPU

In [79]:
nb_merge[nb_merge["Model"] != np.nan]

,Unnamed: 0,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price_euros,Model,Benchmark
0,3,HP,250 G6,Notebook,15.6,Full HD 1920x1080,Core i5 7200U,8GB,256GB SSD,Intel HD Graphics 620,No OS,1.86kg,575.00,Core i5 7200U,57.6
1,11,HP,250 G6,Notebook,15.6,1366x768,Core i5 7200U,4GB,500GB HDD,Intel HD Graphics 620,No OS,1.86kg,393.90,Core i5 7200U,57.6
2,39,HP,250 G6,Notebook,15.6,1366x768,Core i5 7200U,4GB,1TB HDD,Intel HD Graphics 620,Windows 10,1.86kg,488.69,Core i5 7200U,57.6
3,43,Asus,X542UQ-GO005 (i5-7200U/8GB/1TB/GeForce,Notebook,15.6,1366x768,Core i5 7200U,8GB,1TB HDD,Nvidia GeForce 940MX,Linux,2.3kg,522.99,Core i5 7200U,57.6
4,49,Dell,Inspiron 3567,Notebook,15.6,Full HD 1920x1080,Core i5 7200U,4GB,256GB SSD,AMD Radeon R5 M430,Windows 10,2.3kg,639.00,Core i5 7200U,57.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1055,954,HP,Omen 17-W006na,Gaming,17.3,IPS Panel Full HD 1920x1080,Core i5 6300HQ,8GB,128GB SSD + 1TB HDD,Nvidia GeForce GTX 965M,Windows 10,2.75kg,1399.00,Core i5 6300HQ,64.1
1056,1091,Lenovo,Ideapad 700-15ISK,Notebook,15.6,IPS Panel Full HD 1920x1080,Core i5 6300HQ,4GB,1TB HDD,Nvidia GeForce GTX 950M,Windows 10,2.3kg,977.00,Core i5 6300HQ,64.1
1057,1144,HP,Omen -,Gaming,17.3,IPS Panel Full HD 1920x1080,Core i5 6300HQ,8GB,128GB SSD + 1TB HDD,Nvidia GeForce GTX 1060,Windows 10,3.35kg,1129.00,Core i5 6300HQ,64.1
1058,1172,Dell,XPS 15,Notebook,15.6,IPS Panel Touchscreen / 4K Ultra HD 3840x2160,Core i5 6300HQ,8GB,256GB SSD,Nvidia GeForce 960M,Windows 10,2.04kg,2250.68,Core i5 6300HQ,64.1


### GPU Benchmarks

In [98]:
#gpu.sample(5)
nb["Gpu"].head()

0     Iris Plus Graphics 640
1           HD Graphics 6000
2            HD Graphics 620
3             Radeon Pro 455
4     Iris Plus Graphics 650
Name: Gpu, dtype: object

In [99]:
from difflib import get_close_matches 
#gpu[gpu['Model'].str.contains('620')].head()
matches = nb["Gpu"].apply(lambda x: get_close_matches(x, gpu["Model"]))

0       [Iris Plus G4, Iris Plus G7, UHD Graphics 605]
1    [UHD Graphics 600, HD Graphics P4600, UHD Grap...
2    [UHD Graphics 605, UHD Graphics 600, HD Graphi...
3    [Radeon Pro 5500M, Radeon Pro 5300M, Radeon HD...
4       [Iris Plus G7, Iris Plus G4, UHD Graphics 750]
Name: Gpu, dtype: object

In [101]:
match = matches.apply(lambda x: x[0] if len(x) > 0 else "")
nb["Approx Gpu"] = match

In [92]:
nb['Gpu'] = nb['Gpu'].astype('str')
gpu["Model"] = gpu["Model"].astype('str')
nb["Gpu"] = nb["Gpu"].apply(lambda x: re.sub("AMD|Intel|([\d\.]+GHz)|(Dual Core)|(Quad Core)|", "", x.strip()))
gpu["Model"] = gpu["Model"].apply(lambda x: re.sub("-", " ", x.strip()))